In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

# conf = SparkConf()
# conf.set('spark.jars.packages', "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")
# spark = SparkSession.builder.appName("multi").config(conf=conf).getOrCreate()

spark = SparkSession \
    .builder \
    .appName("multi") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","datalake") \
    .config("spark.mongodb.input.collection", "i") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

sc =spark.sparkContext



In [2]:
'''
'USE_DT': '20210806',
'LINE_NUM': '1호선',
'SUB_STA_NM': '종각',
'RIDE_PASGR_NUM': 27789.0,
'ALIGHT_PASGR_NUM': 26895.0,
'WORK_DT': '20210809'
'''  

subwaySchema =  StructType([
    StructField("USE_DT", StringType(),True),
    StructField("LINE_NUM", StringType(),True),
    StructField("SUB_STA_NM", StringType(),True),
    StructField("RIDE_PASGR_NUM", IntegerType(),True),
    StructField("ALIGHT_PASGR_NUM", IntegerType(),True),
    StructField("WORK_DT", StringType(),True),
  ])

In [3]:
# df = spark.read.format("mongo").load()
df = spark.read.schema(subwaySchema).format("com.mongodb.spark.sql.DefaultSource").load() 

In [5]:
df.limit(100).show()

+--------+--------+----------------------+--------------+----------------+--------+
|  USE_DT|LINE_NUM|            SUB_STA_NM|RIDE_PASGR_NUM|ALIGHT_PASGR_NUM| WORK_DT|
+--------+--------+----------------------+--------------+----------------+--------+
|20190101|   1호선|                서울역|         39420|           31121|20190104|
|20190101|   1호선|                  시청|         11807|           10322|20190104|
|20190101|   1호선|                  종각|         20944|           16658|20190104|
|20190101|   1호선|               종로3가|         17798|           15762|20190104|
|20190101|   1호선|               종로5가|         13578|           13282|20190104|
|20190101|   1호선|                동대문|          9337|           10457|20190104|
|20190101|   1호선|                신설동|          6832|            6930|20190104|
|20190101|   1호선|                제기동|         10187|           10178|20190104|
|20190101|   1호선|청량리(서울시립대입구)|         15007|           15397|20190104|
|20190101|   1호선|                동묘앞|     

In [6]:
pipeline = "{'$match': {'USE_DT': '20210806'}}"
df = spark.read.format("mongo").option("pipeline", pipeline).load()
 
df.show()

+----------------+--------+--------------+-----------------------+--------+--------+--------------------+
|ALIGHT_PASGR_NUM|LINE_NUM|RIDE_PASGR_NUM|             SUB_STA_NM|  USE_DT| WORK_DT|                 _id|
+----------------+--------+--------------+-----------------------+--------+--------+--------------------+
|         34595.0|   1호선|       34834.0|                 서울역|20210806|20210809|{61120261d3f97124...|
|          6686.0|   1호선|        6432.0|                 동묘앞|20210806|20210809|{61120261d3f97124...|
|         17720.0|   1호선|       17909.0|                   시청|20210806|20210809|{61120261d3f97124...|
|         26895.0|   1호선|       27789.0|                   종각|20210806|20210809|{61120261d3f97124...|
|         20195.0|   1호선|       21808.0|                종로3가|20210806|20210809|{61120261d3f97124...|
|         18856.0|   1호선|       19260.0|                종로5가|20210806|20210809|{61120261d3f97124...|
|          9261.0|   1호선|       10061.0|                 동대문|20210806|2021